# Bedrock model integration with Langchain Agents

## Architecture
![](./images/arch-agents.png)

## Setup

In [1]:
!pip install langchain wikipedia!pip install numexpr

In [2]:
import json
import os
import sys
import boto3

# ---- ⚠️ Un-comment and edit the below lines as needed for your AWS setup ⚠️ ----

# os.environ["AWS_DEFAULT_REGION"] = "<REGION_NAME>"  # E.g. "us-east-1"
# os.environ["AWS_PROFILE"] = "<YOUR_PROFILE>"
# os.environ["BEDROCK_ASSUME_ROLE"] = "<YOUR_ROLE_ARN>"  # E.g. "arn:aws:..."

model_parameter = {"temperature": 0.0, "top_p": .5, "max_tokens_to_sample": 2000}

## Using ReAct: Synergizing Reasoning and Acting in Language Models Framework

The ReAct framework could enable large language models to interact with external tools to obtain additional information that results in more accurate and fact-based responses.

In [3]:
from langchain.agents import load_tools
from langchain.agents import initialize_agent, Tool
from langchain.agents import AgentType
from langchain.llms.bedrock import Bedrock
from langchain import LLMMathChain
from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper

### Setup the LangChain chains

In [4]:
react_agent_llm = Bedrock(model_id="anthropic.claude-instant-v1", model_kwargs=model_parameter)
math_chain_llm = Bedrock(model_id="anthropic.claude-instant-v1",
                         model_kwargs={"temperature":0,"stop_sequences" : ["```output"]})

tools = load_tools(["wikipedia"], llm=react_agent_llm)

llm_math_chain =  LLMMathChain.from_llm(llm=math_chain_llm, verbose=True)

llm_math_chain.llm_chain.prompt.template = """Human: Given a question with a math problem, provide only a single line mathematical expression that solves the problem in the following format. Don't solve the expression only create a parsable expression.
```text
${{single line mathematical expression that solves the problem}}
```

Assistant:
 Here is an example response with a single line mathematical expression for solving a math problem:
```text
37593**(1/5)
```

Human: {question}

Assistant:"""

tools.append(
    Tool.from_function(
        func=llm_math_chain.run,
        name="Calculator",
        description="Useful for when you need to answer questions about math.",
    )
)

/opt/conda/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The class `Bedrock` was deprecated in LangChain 0.0.34 and will be removed in 0.3. An updated version of the class exists in the langchain-aws package and should be used instead. To use it run `pip install -U langchain-aws` and import as `from langchain_aws import BedrockLLM`.
  warn_deprecated(


### Setup the ReAct agent

In [5]:
react_agent = initialize_agent(tools, 
                               react_agent_llm, 
                               agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, 
                               verbose=True,
                            #    max_iteration=2,
                            #    return_intermediate_steps=True,
                            #    handle_parsing_errors=True,
                               )

/opt/conda/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The function `initialize_agent` was deprecated in LangChain 0.1.0 and will be removed in 0.3.0. Use Use new agent constructor methods like create_react_agent, create_json_agent, create_structured_chat_agent, etc. instead.
  warn_deprecated(


### Setup the prompt template

In [6]:
prompt_template = """Use the following format:
Question: the input question you must answer
Thought: you should always think about what to do, Also try to follow steps mentioned above
Action: the action to take, should be one of ["wikipedia", "Calculator"]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Question: {input}

Assistant:
{agent_scratchpad}"""

react_agent.agent.llm_chain.prompt.template=prompt_template

### Ask a question (for testing)

In [7]:
question = "What is Amazon SageMaker? What is the launch year multiplied by 2"
react_agent(question)

/opt/conda/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(




> Entering new AgentExecutor chain...
 Here is the response in the specified format:

Question: What is Amazon SageMaker? What is the launch year multiplied by 2

Thought: Let me first look up what Amazon SageMaker is on Wikipedia

Action: wikipedia 

Action Input: Amazon SageMaker

Observation: Page: Amazon SageMaker
Summary: Amazon SageMaker is a cloud-based machine-learning platform that allows the creation, training, and deployment by developers of machine-learning (ML) models on the cloud. It can be used to deploy ML models on embedded systems and edge-devices. The platform was launched in November 2017.

Page: Timeline of Amazon Web Services
Summary: This is a timeline of Amazon Web Services, which offers a suite of cloud computing services that make up an on-demand computing platform.

Page: GPT-J
Summary: GPT-J or GPT-J-6B is an open-source large language model (LLM) developed by EleutherAI in 2021. As the name suggests, it is a generative pre-trained transformer model design

{'input': 'What is Amazon SageMaker? What is the launch year multiplied by 2',
 'output': 'Amazon SageMaker is a cloud-based machine learning platform launched by AWS in 2017. The launch year multiplied by 2 is 4034.'}

---

## Generating the tool code

### Build the prompt, invoke the model, and register the tool

In [8]:
import xml.etree.ElementTree as ET
from IPython.display import display, Markdown, Latex

boto3_bedrock = boto3.client(service_name='bedrock-runtime',
                             region_name=os.environ["AWS_DEFAULT_REGION"])

prompt_data = """
Human: You are an AI python code generator. You write really great code.

Write a python function named list_tagged_instances with one parameter named tagname. 

The function queries the boto3 library to return a list all of the EC2 instances using boto3 resource that have a the tag "Name" equal to the tagname parameter. 

return the code inside <code></code>.
Assistant:"""

body = json.dumps({"prompt": prompt_data, "max_tokens_to_sample": 500})
modelId = "anthropic.claude-instant-v1"  
accept = "application/json"
contentType = "application/json"

response = boto3_bedrock.invoke_model(
    body=body, modelId=modelId, accept=accept, contentType=contentType
)
response_body = json.loads(response.get("body").read())

tree = ET.ElementTree(ET.fromstring(response_body.get("completion")))
python_code = tree.getroot().text

display(Markdown(f'```{python_code}```'))

exec(python_code)

tools = [
    Tool.from_function(
        func=list_tagged_instances,
        name="List",
        description="Queries the boto3 library to return a list all of the EC2 instances that have the tag \"Name\" equal to the tagname parameter."
    ),
]

```
import boto3

def list_tagged_instances(tagname):
    ec2 = boto3.resource('ec2')
    instances = []
    
    for instance in ec2.instances.all():
        for tag in instance.tags:
            if tag['Key'] == 'Name' and tag['Value'] == tagname:
                instances.append(instance)
                
    return instances
```

### Test the code
If the code above looks reasonable, we can use it to run our agent.

In [9]:
llm = Bedrock(model_id="anthropic.claude-instant-v1", client=boto3_bedrock, model_kwargs=model_parameter)

react_agent = initialize_agent(tools, llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True)

question = """Human: Please list my EC2 instances with the tag delete.
Assistant:"""

result = react_agent.run(question)

print(f"{result}")

/opt/conda/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(




> Entering new AgentExecutor chain...
 I need to use the List action to get instances tagged with "delete"
Action: List
Action Input: delete
Observation: []
Thought: There are no instances tagged with "delete"
Final Answer: There are no EC2 instances with the tag "delete".

> Finished chain.
There are no EC2 instances with the tag "delete".


## Another variation - Search for an instance that does not exist
The prompt changes to search for EC2 instances with a tag that does not exist, so no instances should be returned.

In [10]:
question = """Human: Please list my EC2 instances with the tag doesnotexist.
Assistant:"""

result = react_agent(question)

print(f"{result}")



> Entering new AgentExecutor chain...
 I need to check if any instances have the tag "doesnotexist"
Action: List
Action Input: doesnotexist
Observation: []
Thought: No instances were returned, so there are no instances with that tag
Final Answer: There are no EC2 instances with the tag "doesnotexist".

> Finished chain.
{'input': 'Human: Please list my EC2 instances with the tag doesnotexist.\nAssistant:', 'output': 'There are no EC2 instances with the tag "doesnotexist".'}


---
## Database Tools

### Build the simulated database table and custom tool

In [11]:
customer_table=[
  {
    "id": 1, 
    "first_name": "John", 
    "last_name": "Doe",
    "age": 35,
    "postal_code": "90210"
  },
  {  
    "id": 2,
    "first_name": "Jane",
    "last_name": "Smith", 
    "age": 27,
    "postal_code": "12345"
  },
  {
    "id": 3, 
    "first_name": "Bob",
    "last_name": "Jones",
    "age": 42,
    "postal_code": "55555"
  },
  {
    "id": 4,
    "first_name": "Sara", 
    "last_name": "Miller",
    "age": 29, 
    "postal_code": "13579"
  },
  {
    "id": 5,
    "first_name": "Mark",
    "last_name": "Davis",
    "age": 31,
    "postal_code": "02468"
  },
  {
    "id": 6,
    "first_name": "Laura",
    "last_name": "Wilson",
    "age": 24,
    "postal_code": "98765" 
  },
  {
    "id": 7,
    "first_name": "Steve",
    "last_name": "Moore",
    "age": 36,
    "postal_code": "11223"
  },
  {
    "id": 8,
    "first_name": "Michelle",
    "last_name": "Chen",
    "age": 22,
    "orders": [
        {
            "order_id": 1,
            "description": "An order of 1 dozen pencils"
        },
        {
            "order_id": 2,
            "description": "An order of 2 markers"
        }
    ],
    "postal_code": "33215"
  },
  {
    "id": 9,
    "first_name": "David",
    "last_name": "Lee",
    "age": 29,
    "postal_code": "99567"
  },
  {
    "id": 10,
    "first_name": "Jessica",
    "last_name": "Brown",
    "age": 18, 
    "postal_code": "43210"
  }
]

def customer_lookup(id):
    print(f"search by customer {id}")
    for customer in customer_table:
        if customer["id"] == int(id):
            print(f"found customer {id} {customer}")
            return customer
        
    return None

tools.append(Tool.from_function(
        name="CustomerLookup",
        func=customer_lookup,  # Mock Function, replace with an api call
        description="Use this when you need to lookup a customer by id."
    ))

### Lookup customer information

In [12]:
react_agent = initialize_agent(tools, llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True)

question = """\n\nHuman: write one sentence summary about the information you know about the customer with an id of 2.

\n\nAssistant: Here is the one sentence summary: """

result = react_agent.run(question)

print(f"{result}")



> Entering new AgentExecutor chain...
 I need to lookup the customer by id
Action: CustomerLookup
Action Input: 2search by customer 2
found customer 2 {'id': 2, 'first_name': 'Jane', 'last_name': 'Smith', 'age': 27, 'postal_code': '12345'}

Observation: {'id': 2, 'first_name': 'Jane', 'last_name': 'Smith', 'age': 27, 'postal_code': '12345'}
Thought: I now know the customer details
Final Answer: Jane Smith who is 27 years old lives at postal code 12345.

> Finished chain.
Jane Smith who is 27 years old lives at postal code 12345.
